### Using FISSA with Suite2P, AFTER merging the .tif files in each folder.
Suite2P is a popular toolbox for cell detection and signal extraction. 

Here we illustrate how one can use the ROIs and motion corrected videos generated detected by Suite2P, and use FISSA to extract and decontaminate the traces.

For more information about the Suite2P toolbox see: 
https://www.biorxiv.org/content/10.1101/061507v2
https://mouseland.github.io/suite2p/_build/html/index.html
https://science.sciencemag.org/content/364/6437/eaav7893.full?ijkey=PHaXxFoO8kMGw&keytype=ref&siteid=sci


#### Imports

In [ ]:
import holoviews as hv
%load_ext holoviews.ipython
hv.extension('matplotlib')
%matplotlib inline

# file operations
import glob
from scipy.io import loadmat

# FISSA
import fissa

# numpy
import numpy as np

# os / re / tqdm
import os # directory tool
import re # regular expressions
import tqdm.notebook as tqdm # progress bar
from PIL import Image # Read tiff

import warnings # Stops annoying warning messages
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

import imagej
ij = imagej.init('net.imagej:imagej:2.1.0', headless=False)

Set the root path that contains folders you want to analyze

In [ ]:
# change the folder location accordingly
path = r'Y:\Jui-Yen Huang\Derived Files\Data\In vivo Calcium Imaging\20220522_0601_jRGECO1a_C57-C6-2\20220601_jRGECO1a_C57-C6-2_P21'

First, define the ImageJ macros to be used by pyimagej to merge the tiffs and save them.

In [ ]:
openfiles = """
    #@ String input
    setBatchMode("show")
    list = getFileList(input);
    count = list.length;
    for (i = 0; i < count; i++){
        if (endsWith(list[i], ".tif")) {
            open(input + list[i]);
    }
    }   
"""
concat_and_save = """
#@ String input
output= input + "../../../concatenate/";
list = getFileList(input);
count = list.length;
command = "open ";
idx = 1;
for (i = 0; i < count; i++) {
    if (endsWith(list[i], ".tif")) {
        command = command + "image" + idx + "=" + list[i] + " ";
        idx = idx + 1;
}
}
run("Concatenate...", command);
saveAs("Tiff", output + "concatenate.tif");
run("Close")
"""

Now, we define a function to process one data folder. This involves either loading the tiff file in, or setting up the required paths to access the data. Then the Fissa experiment is run, and the outputs are saved.

In [ ]:
%%opts Curve {+axiswise}
def process_one(path, ncores_preparation=None, ncores_separation=None, 
                nRegions=4, expansion=0.3, alpha=0.2, ncells=5,
                redo_prep=True, redo_sep=True, load_tiff_manually=False):
    '''
    Run the Suite2P analysis on one folder of data
    Args:
        - path: The root path, or the path of the litter folder that contains subfolders of each animal
        - ncores_prep/sep: Number of cores to use for the prep/sep steps of the analysis
            - Default -> None, which uses all available cores
        - regions: Number of subregions FISSA will use for the neuropil region (default=4)
        - expansion: Normally the area of each subregion is set the same as the central ROI; default is 1 in FISSA; (default=0.3)
        - alpha: The degree of signal sparsity; default is 0.1 in FISSA; (default=0.2)
        - ncells: Number of cell ROIs we want to use with FISSA
        - redo_prep/sep: Whether to redo the prep/sep steps of the analysis or load previous output (default=True/True)
    '''
    if not os.path.exists(path + '/concatenate'):
        os.mkdir(path + '/concatenate')
    curr_folder = path.split('\\')[-1]
    print(f'Running analysis on {curr_folder}')
    print('Merging .tif files')
    args = {'input': path + '/suite2p/plane0/reg_tif/'}
    ij.py.run_macro(openfiles, args)
    ij.py.run_macro(concat_and_save, args)
    stat = np.load(path + '/suite2p/plane0/stat.npy', allow_pickle=True) # cell stats
    ops = np.load(path + '/suite2p/plane0/ops.npy', allow_pickle=True).item() # options
    iscell = np.load(path + '/suite2p/plane0/iscell.npy', allow_pickle=True)[:, 0] # taking the first column.
    Lx = ops['Lx']
    Ly = ops['Ly']
    ncells = len(stat)
    cell_ids = np.arange(0, ncells) # this gives each cell a number. Edna: the ID start from 0 and so on.
    cell_ids = cell_ids[iscell == 1] # only take the ones that are actually cells.
    num_rois = len(cell_ids)

    # Get the information of Image size and correct ROI ID
    # np.zeros generate numpy array
    ROIS = [np.zeros((Ly, Lx), dtype=bool) for n in range(num_rois)]

    # Generate ROI boolean areas
    for i, n in enumerate(cell_ids): # with enumerate i is the position in cell_ids, and n is the actual cell number
        ypix = stat[n]['ypix'][~stat[n]['overlap']]
        xpix = stat[n]['xpix'][~stat[n]['overlap']]
        ROIS[i][ypix,xpix] = 1
    
    # Set input/output paths for data
    output_folder = path + '/FISSA_Suite2P'
    tiff_folder = path + '/concatenate'
    
    # Use Image to open tiff if necessary
    if load_tiff_manually:
        print('Loading Tiff Manually')
        dataset = Image.open(os.path.join(path, 'concatenate.tif'))

        h,w = np.shape(dataset)
        tiffarray = np.zeros((dataset.n_frames,h,w), dtype=np.uint16)
        
        print(f'\n Reading {path}/concatenate/concatenate.tif into memory')
        for i in tqdm.tqdm(list(range(dataset.n_frames)), position=0, leave=True):
            dataset.seek(i)
            tiffarray[i,:,:] = np.array(dataset, dtype=np.uint16)

        dataset.close()
        
        # Create FISSA experiment with loaded array
        exp = fissa.Experiment([tiffarray],[ROIS[:ncells]], output_folder, 
                               nRegions=nRegions, expansion=expansion, alpha=alpha,
                               ncores_preparation=ncores_preparation,
                               ncores_separation=ncores_separation)
    else:
        # Create FISSA experiment using the path to tiff files
        exp = fissa.Experiment(tiff_folder,[ROIS[:ncells]], output_folder, 
                               nRegions=nRegions, expansion=expansion, alpha=alpha,
                               ncores_preparation=ncores_preparation,
                               ncores_separation=ncores_separation)
    
    # Run the seperation
    exp.separate(redo_prep=redo_prep, redo_sep=redo_sep)
    
    # Save the data
    exp.save_to_matlab()
    
    # Delete concatenated .tif files and .ome files
    patterns = ["*ntitled.tif", "*oncatenate.tif", "*.ome*"]
    for pattern in patterns:
        pathname = path + "/**/" + pattern
        files = glob.glob(pathname, recursive=True)
        if not files:
#             print(f'\n Did not find a file matching {pattern} for {directory} \n')
            continue
        else:
            for file in files:
                print(f'Deleting {file}')
                os.remove(file)
    
    os.rmdir(path + '/concatenate')

Now we must loop through the folders within the path and run the process_one function on each subfolder of animal data

In [ ]:
# Get a list of files (and folders) in the path
files_in_path = os.listdir(path)

# Now select only the actual folders within our path
subdirectories = [file for file in files_in_path if os.path.isdir(os.path.join(path, file))]

# Compile a regular expression rule that searches for the format 'P##'
r = re.compile('P([0-9])\w+_')

# Now we select the subdirectories we are interested in if they match the P## criteria
subdirectories_of_interest = [subdirectory for subdirectory in subdirectories if r.match(subdirectory) is not None]

# Run the loop --- tqdm just makes the progress bar
for folder in tqdm.tqdm(subdirectories_of_interest, position=0, leave=True):
    # The r'{}'.format(path) line just makes sure that the path is an 'r' string
    process_one(r'{}'.format(os.path.join(path, folder)))
    print('\n') # Print a blank line to separate outputs